## Here is the code for linking each event with it's main genre

In [3]:
import numpy as np
import ArtistExtractor as AE
import pandas as pd
import glob
import os
import LocationUtils as LU
from ExtractGenreClubs import SplitLineup
from ast import literal_eval
import logging
import desaggregate
import moreFunction

TabError: inconsistent use of tabs and spaces in indentation (moreFunction.py, line 27)

In [ ]:
def printToLog(str,filename="Log/ArtistAndEventGenre_merging.txt"):
    f = open(filename)
    lines = f.readlines()
    
    print(lines)

In [278]:
def mergeEventsWithGenres(eventsdf_filename,artists_filename,dictionnaryOfGenres_filename,encoding = "utf-8"):
    print("---BEGIN MERGE----")
    print("This script takes approximately one hour to perform")
    events = pd.read_csv(eventsdf_filename, encoding="utf-8")
    artists = pd.read_csv(artists_filename, encoding="utf-8")
    dictionnaryOfGenres = LU.loadDictionary(dictionnaryOfGenres_filename,enc="utf-8")
    
    dataframe_merged = events.copy()
    
    
    #Getting columns
    columnsArtists = []
    columnsEvents = []
    for c in artists.columns:
        if("Unnamed" not in c):
            columnsArtists.append(c)
    
    for c in events.columns:
        if("Unnamed" not in c):
            columnsEvents.append(c)
           
    logging.basicConfig(filename='LOG/merging_events_and_artists.log',level=logging.DEBUG)
    
    i =0
    for id, row in dataframe_merged.iterrows():
        logging.info("-------------------------------")
        i+=1
        #if(i>100000):
         #   break
        if(i%1000==0):
            print(i)
            #Saving actual state
            artists = artists[columnsArtists]
            events = events[columnsEvents]
            artists.to_csv(artists_filename.replace(".csv","_merged.csv"),encoding=encoding)
            events.to_csv(eventsdf_filename.replace(".csv","_merged.csv"),encoding=encoding)
        
        #getting genre
        genre = str(row["genre"])
        #getting artist
        lineup = literal_eval(moreFunctionrow["artists"])
        logging.info(lineup)
        
        #looking for corresponding genre in dictionnary:
        maingenre = dictionnaryOfGenres.get(genre.lower())

        logging.info("event genre is: "+str(genre)+"->"+str(maingenre))

        maxGenre = None
        #check if artists has a genre to complete artists database
        if(maingenre!="None" and maingenre!=None and maingenre in dictionnaryOfGenres):
            for a in lineup:
                
                #Getting value
                vals = artists[artists["artist"] == a.strip()]["genre"].values
                genre_artist = "None"
                if(len(vals)>0):
                    genre_artist = str(vals[0])
                    
                #Setting artist genre
                if(genre_artist=="nan" or genre_artist=="None"):
                    #set genre of artist to genre of event and convert genre in dataframe
                    artists.loc[artists["artist"] == a.strip(),"genre"] = maingenre.lower()
                    maxGenre = maingenre.lower()
                    logging.info(str(a)+"<-"+maingenre.lower())
        
        genres_in_lineup = []
        
        for a in lineup:
            
            #Getting value
            vals = artists[artists["artist"] == a.strip()]["genre"].values
            genre_artist = "None"
            if(len(vals)>0):
                genre_artist = str(vals[0])
                
            #Getting corresponding maingenre for this artist   
            if(genre_artist!="nan" and genre_artist!="None" and genre_artist!=None):
                genres_in_lineup.append(genre_artist)
            
        #getting most representative genre among all artists
        maxGenre = AE.getMaxGenre(genres_in_lineup)
        
        #If an artist has a genre, then we set the genre of event to this artist genre
        if(maxGenre!=None):
            logging.info("From lineup : "+str(genres_in_lineup)+"->"+str(maxGenre[0]))
            if(maxGenre[0] in dictionnaryOfGenres):
                events.loc[id,"genre"] = maxGenre[0].lower()
            else:
                #Not in dictionnary
                events.loc[id,"genre"] = None
        else:
            events.loc[id,"genre"] = None
    
    artists = artists[columnsArtists]
    events = events[columnsEvents]
    artists.to_csv(artists_filename.replace(".csv","_merged.csv"),encoding=encoding)
    events.to_csv(eventsdf_filename.replace(".csv","_merged.csv"),encoding=encoding)
    
    return events,artists

In [288]:
def extendingGenres(artists_df,dictionnaryOfGenres,debug=False):
    genres = list(set(dictionnaryOfGenres.values()))
    
    #Extending
    #Getting columns
    columnsArtists = []
    for c in artists_df.columns:
        if("Unnamed" not in c):
            columnsArtists.append(c)
    columnsArtists = columnsArtists+genres+["total_genres"]

    print("Extending columns to these genres :")
    print(columnsArtists)
    
    i=0
    for id,row in artists_df.iterrows():
        i+=1
        if(debug and i%3000==0):
            print(i)
        main_genres = row["main_genres"]
        
        
        total = 0.0
        
        if(main_genres!=None and main_genres!="None"):
            total = len(literal_eval(main_genres))
            
            #Check if artist has a genre (from other dataframes)
            if(total==0):
                if(raw["genre"]!=None and raw["genre"]!="None"):
                    main_genres = list(literal_eval(raw["genre"]))
                    artists_df.loc[id,"main_genres"] = main_genres
                    total = 1.0
                
        artists_df.loc[id,"total_genres"]=total
        
        #percententage of this genre
        for c in genres:
            if(total==0):
                 artists_df.loc[id,c] = 0.0
            else:
                artists_df.loc[id,c] = main_genres.count(c)/total

    return artists_df

In [279]:
eventsdf_filename = "FullData/EventsAndClubs.csv"
artists_filename = "FullData/ArtistDF_withGenres.csv"
dictionnaryOfGenres_filename = "FullData/AllGenresDic"

events,artists = mergeEventsWithGenres(eventsdf_filename,artists_filename,dictionnaryOfGenres_filename)


---BEGIN MERGE----
This script takes approximately one hour to perform
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
Extending genres of artists..
['artist', 'genres_spotify', 'genres_ra', 'main_genres', 'top3_genres', 'genre', 'all_genres', 'avant-garde', 'latin', 'african', 'r&b and soul', 'jazz', 'classic', 'folklore', 'orchestral', 'hip hop', 'comedy', 'easy listening', 'country', 'blues', 'asian', 'electronic', 'rock', 'pop', 'folk', 'caribbean and caribbean-influenced', 'total_genres']
Andy Gruber Band : None->0
DJ Raykard : None->0
Martin Sander : None->0
Bernhard Hörler : None->0
Oomph : ['rock', 'rock', 'rock', 'rock', 'electronic', 'jazz', 'latin', 'rock', 'rock', 'rock', 'electronic', 'pop', 'latin', 'hip hop', 'electronic', 'rock', 'rock']->17
Plusmacher : ['hip hop', 'hip hop']->2
Konstantin Scherbakov 

In [289]:
print("Extending genres of artists..")

dictionnaryOfGenres_filename = "FullData/AllGenresDic"
dictionnaryOfGenres = LU.loadDictionary(dictionnaryOfGenres_filename,enc="utf-8")
artists_filename = "FullData/ArtistDF_withGenres_merged.csv"
artists = pd.read_csv(artists_filename,encoding="utf-8")

artists = extendingGenres(artists,dictionnaryOfGenres,True)
artists.to_csv(artists_filename.replace(".csv","_merged_ext.csv"),encoding="utf-8")
print("Done")

Extending genres of artists..
Extending columns to these genres :
['artist', 'genres_spotify', 'genres_ra', 'main_genres', 'top3_genres', 'genre', 'all_genres', 'avant-garde', 'latin', 'african', 'r&b and soul', 'jazz', 'classic', 'folklore', 'orchestral', 'hip hop', 'comedy', 'easy listening', 'country', 'blues', 'asian', 'electronic', 'rock', 'pop', 'folk', 'caribbean and caribbean-influenced', 'total_genres', 'avant-garde', 'latin', 'african', 'r&b and soul', 'jazz', 'classic', 'folklore', 'orchestral', 'hip hop', 'comedy', 'easy listening', 'country', 'blues', 'asian', 'electronic', 'rock', 'pop', 'folk', 'caribbean and caribbean-influenced', 'total_genres']
3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
Done


In [287]:
print("Deaggregatin data..")
deaggr = desaggregate.desaggregate(events,False)
print("Saving..")
deaggr.to_csv(artists_filename.replace(".csv","_merged_deaggr.csv"),encoding="utf-8")
print("Done")

Deaggregatin data..
Saving..
Done


In [275]:
artists_filename = "FullData/ArtistDF_withGenres.csv"
dictionnaryOfGenres_filename = "FullData/AllGenresDic"
dictionnaryOfGenres = LU.loadDictionary(dictionnaryOfGenres_filename,enc="utf-8")
artists_df = pd.read_csv(artists_filename)

artists = extendingGenres(artists_df,dictionnaryOfGenres)
artists.to_csv("FullData/ArtistDF_withGenres_ext.csv")

['artist', 'genres_spotify', 'genres_ra', 'main_genres', 'top3_genres', 'genre', 'all_genres', 'avant-garde', 'latin', 'african', 'r&b and soul', 'jazz', 'classic', 'folklore', 'orchestral', 'hip hop', 'comedy', 'easy listening', 'country', 'blues', 'asian', 'electronic', 'rock', 'pop', 'folk', 'caribbean and caribbean-influenced', 'total_genres']


0         0.0
1         0.0
2         0.0
3         0.0
4        17.0
5         2.0
6         3.0
7         2.0
8         0.0
9         0.0
10        0.0
11        0.0
12        2.0
13        0.0
14        0.0
15        0.0
16        0.0
17        0.0
18       20.0
19       15.0
20        NaN
21        NaN
22        NaN
23        NaN
24        NaN
25        NaN
26        NaN
27        NaN
28        NaN
29        NaN
         ... 
45348     NaN
45349     NaN
45350     NaN
45351     NaN
45352     NaN
45353     NaN
45354     NaN
45355     NaN
45356     NaN
45357     NaN
45358     NaN
45359     NaN
45360     NaN
45361     NaN
45362     NaN
45363     NaN
45364     NaN
45365     NaN
45366     NaN
45367     NaN
45368     NaN
45369     NaN
45370     NaN
45371     NaN
45372     NaN
45373     NaN
45374     NaN
45375     NaN
45376     NaN
45377     NaN
Name: total_genres, dtype: float64